# Анализ и восстановление грейда

In [86]:
import pandas as pd
import plotly.express as px
import hvplot.pandas

In [87]:
pd.read_csv('../data/processed/vacancies.csv', encoding='utf-8').columns

Index(['description', 'vacancy_id', 'employer', 'name', 'salary_row', 'salary',
       'salary_from', 'salary_to', 'experience', 'schedule', 'skills',
       'address', 'url', 'query', 'publish_city_str', 'employment_type',
       'employment_workhours', 'publish_date', 'city', 'city_rating'],
      dtype='object')

In [105]:
df = pd.read_csv('../data/processed/vacancies.csv', encoding='utf-8')\
    [['vacancy_id','name', 'salary', 'salary_from', 'salary_to', 'experience', 'description']]
profset_df = pd.read_csv('../data/features/vacancy_profset.csv')

df = df.merge(profset_df, on='vacancy_id')
df['prof_set'] = df.prof_set.apply(lambda x: set([y.strip(" '") for y in x.strip('{}').split(',')]))

df.head()

,vacancy_id,name,salary,salary_from,salary_to,experience,description,prof_set
0,80485628,Аналитик,True,54350.0,54350.0,не требуется,"""DSM Group"" – признанный эксперт и надёжный па...",{Аналитик}
1,80652370,Игровой аналитик (продуктовый аналитик),False,NaN,NaN,1–3 года,"""Zebomba Games"" — игровая студия-разработчик.М...",{Продуктовый аналитик}
2,78278410,Главный системный аналитик/Senior system Analy...,False,NaN,NaN,3–6 лет,"""Азиатско-Тихоокеанский Банк"" - опорный банк Д...",{Системный аналитик}
3,80585316,Бизнес-аналитик (транзакционные продукты),False,NaN,NaN,1–3 года,"""Азиатско-Тихоокеанский Банк"" - опорный банк Д...",{Бизнес-аналитик}
4,80223554,Маркетолог-аналитик,True,87000.0,174000.0,более 6 лет,"""Анимаго"" - уникальный стартап в сфере детског...",{Аналитик}


## Mini-EDA

In [89]:
df.salary.value_counts(normalize=True, dropna=False)

False    0.704025
True     0.295975
Name: salary, dtype: float64

In [90]:
grade_kw = {
    'Intern' : ['Intern', 'Помощник', 'стажер', 'стажор'],
    'Junior' : ['Junior', 'Начинающий', 'Младший'],
    'Middle' : ['Middle', 'Старший'],
    'Senior' : ['Senior', 'Ведущий'],
    'Lead' : ['Lead', 'Главный', 'Лидер'],
    'Head' : ['Head', 'Руководитель'],
    'Сhief' : ['Сhief', 'Директор']
}

In [93]:
# сколько вакансий упоминают грейд:
bag_of_words = set().union(*[set(x) for x in grade_kw.values()])
df.name.apply(lambda x: any([(w.lower() in x.lower()) for w in bag_of_words])).value_counts(normalize=True)

False    0.776691
True     0.223309
Name: name, dtype: float64

In [114]:
# а если в имени ИЛИ дескрипшине, но в дескрипшине без конфликта грейдов (только  один любой)
bag_of_words = set().union(*[set(x) for x in grade_kw.values()])
df.apply(lambda x: 
                    any([(w.lower() in x['name'].lower()) for w in bag_of_words]) or
                    sum([(w.lower() in x['description'].lower()) for w in bag_of_words]) == 1
            , axis=1). \
        value_counts(normalize=True)

False    0.62493
True     0.37507
dtype: float64

In [115]:
# а если в имени ИЛИ дескрипшине ИЛИ ЗП
# а если в имени ИЛИ дескрипшине, но в дескрипшине без конфликта грейдов (только  один любой)
bag_of_words = set().union(*[set(x) for x in grade_kw.values()])
df.apply(lambda x: 
                    any([(w.lower() in x['name'].lower()) for w in bag_of_words]) or
                    sum([(w.lower() in x['description'].lower()) for w in bag_of_words]) == 1 or
                    x['salary'] == 1
            , axis=1). \
        value_counts(normalize=True)

True     0.561766
False    0.438234
dtype: float64

Итого мы имеем: 
- 22% имеют грейд в названии
- 37% имеют грейд в названии или описании
- 56% имеют ещё и зп, из которой в теории можно восстановить (но не факт)

Тогда посмотрим, на те, что имеют грейд


In [120]:
def get_grade(r):
    # name
    for k, vv in grade_kw.items():
        if any([(v.lower() in r['name'].lower()) for v in vv]):
            return k
    #description
    # grades = []
    # for k, vv in grade_kw.items():
    #     if any([(v.lower() in r['description'].lower()) for v in vv]):
    #         grades.append(k)
    # if len(grades) == 1:
    #     return grades[0]
    
    return None

df['grade'] = df.apply(get_grade, axis=1)
dfg = df[~df.grade.isna()]
dfg['grade_id'] = dfg.grade.apply(lambda x: list(grade_kw.keys()).index(x))
dfg = dfg.sort_values(by='grade_id')
dfg.grade.value_counts()

/var/folders/3c/_r_wzsdn5h3fr5qwhjx2hb440000gn/T/ipykernel_4214/2350629696.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfg['grade_id'] = dfg.grade.apply(lambda x: list(grade_kw.keys()).index(x))


Senior    257
Middle    185
Junior    107
Head       94
Lead       88
Intern     62
Сhief       6
Name: grade, dtype: int64

In [119]:
# Пример того, как грей lead НЕПРАВИЛЬНО выбрался из описания
dfg.loc[1039, 'description']

'Группа компаний LOGISTIX – системный интегратор в сфере логистики в России, инновационный лидер отечественного рынка WMS. О нас:  Более 18 лет мы предоставляем услуги по оптимизации складских комплексов: профессиональный логистический консалтинг, разработку и внедрение адаптируемых систем LEAD; 2 место на российском рынке WMS-решений; 300 + проектов реализовано в области логистики; Сотрудничаем с ведущими российскими и европейскими университетами.  Что мы делаем:  Разрабатываем собственные системы управления профессионального уровня: система управления складом-LEAD WMS, транспортом-LEAD TMS, производством-LEAD MES, закупками и продажами-STOCK&SALES.   Инновации в логистике: технологии виртуальной и дополненной реальности, нейронные сети и искусственный интеллект для анализа данных, биотелеметрия, имитационное моделирование. Экспертный логистический консалтинг.  В связи с расширением штата мы ищем Разработчика баз данных (PostgreSQL): Обязанности:  Разработка функционала уровня БД сист

In [123]:
# какие професии представлены (в штуках)
def prof_value_counts(df):
    all_prof = set().union(*df.prof_set.to_list())
    profname_to_count = {}
    for n in all_prof:
        profname_to_count[n] = df.prof_set.apply(lambda x: n in x).sum()

    return {k: v for k, v in sorted(profname_to_count.items(), key=lambda item: item[1], reverse=True)}

prof_value_counts(dfg)

{'Аналитик': 317,
 'Бизнес-аналитик': 129,
 'Data Scientist': 98,
 'Инженер данных': 87,
 'Системный аналитик': 65,
 'Аналитик данных': 62,
 'ML инженер': 49,
 'Big Data': 40,
 'NLP': 12,
 'Продуктовый аналитик': 11,
 'Администратор баз данных': 11,
 'Computer Vision': 11,
 'Аналитик BI': 5}

In [71]:
# а теперь сколько есть с зп и грейдом и больше 5-и шт
pp = prof_value_counts(dfg[dfg.salary==1])
[pp.pop(k) for k in [k for k, v in pp.items() if v <= 5]]
pp

{'Аналитик': 98,
 'Бизнес-аналитик': 35,
 'Инженер данных': 28,
 'Системный аналитик': 25,
 'Аналитик данных': 20,
 'Data Scientist': 19,
 'ML инженер': 13,
 'Big Data': 6}

In [67]:
dfg['salary_mean'] = (dfg.salary_from + dfg.salary_to) / 2

In [83]:
for p, _ in pp.items():
    #p = list(pp.keys())[0]
    x = dfg[dfg.prof_set.apply(lambda x: p in x)]
    display(x.hvplot.box(y='salary_mean', by='grade', height=400, width=800, 
                order='grade_id', title=p))

:BoxWhisker   [grade]   (salary_mean)

:BoxWhisker   [grade]   (salary_mean)

:BoxWhisker   [grade]   (salary_mean)

:BoxWhisker   [grade]   (salary_mean)

:BoxWhisker   [grade]   (salary_mean)

:BoxWhisker   [grade]   (salary_mean)

:BoxWhisker   [grade]   (salary_mean)

:BoxWhisker   [grade]   (salary_mean)

## Вывод по грейду

Грейд для 22% нормально восстанавливается из названия (из описания нельзя), ещё 20% имеют зп, но она адекватно говорит о грейде в только для некоторых специаьностей и в общем количестве это очень мало

Более сложная модель по тексту пока не вижу целесообразности